<a href="https://colab.research.google.com/github/Khaled2049/CSCI-5832/blob/main/NLP_shared_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install TorchCRF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/en_legal_ner_trf-any-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446.7 MB 21 kB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 6.2 MB 13.8 MB/s 
     |████████████████████████████████| 13.7 MB 47.3 MB/s 
     |████████████████████████████████| 671 kB 68.9 MB/s 
     |████████████████████████████████| 1.1 MB 58.6 MB/s 
     |████████████████████████████████| 53 kB 1.1 MB/s 
     |████████████████████████████████| 4.4 MB 74.7 MB/s 
     |████████████████████████████████| 6.6 MB 55.3 MB/s 
     |████████████████████████████████| 182 kB 41.3 MB/s 
  Attempting uninstall: typer
    Found existing installation: typer 0.7.0
    Uninstalling typer-0.7.0:
      Successfully uninstalled typer-0.7.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2
  Attempting 

In [3]:
!python -m spacy download en_core_web_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 777.4 MB 6.3 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
import requests
import numpy as np
import spacy
import bs4
import urllib3
import json

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
from TorchCRF import CRF as CRFDecoder
from typing import Dict, List, Set, Tuple
import torch

In [6]:
import nltk
from nltk import pos_tag
import pandas as pd
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

# Data Exploration

In [ ]:
split_data = requests.get('https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json', allow_redirects=True)
train = split_data.json()
dev_data = requests.get('https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/dev.json', allow_redirects=True)
# is this supposed to be dev_data.json() ?
dev = split_data.json()

<Response [200]>


In [ ]:
train[0]['annotations'][0]['result']
annotations = pd.DataFrame.from_dict(pd.json_normalize(train[0]['annotations'][0]['result']), orient='columns')
annotations

,id,type,to_name,from_name,value.start,value.end,value.text,value.labels
0,d7a902fe9c23417499a7ef782f9fbdeb,labels,text,label,0,116,"IN THE HIGH COURT OF KARNATAKA,\n ...",[PREAMBLE]
1,8d41599e98424d9480c25109556a7d14,labels,text,label,116,678,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,[PREAMBLE]
2,e501424117da40a7935c2d9f2fb2fe38,labels,text,label,678,964,This Criminal Appeal is filed under Section 37...,[PREAMBLE]
3,4825806388fe43d39f73354b10b5b32d,labels,text,label,964,1093,\n\n This appeal coming on for hearing t...,[PREAMBLE]
4,d6893a25f82948f8be17fc9e876fb716,labels,text,label,1093,1180,\n Heard the learned Counsel for the app...,[NONE]
...,...,...,...,...,...,...,...,...
86,375bebe6234540bf92865406f84bece0,labels,text,label,16196,16241,The judgment of the court below is set aside.,[RPC]
87,70cb6f740b1746259612a99bc2df8062,labels,text,label,16242,16269,The appellant is acquitted.,[RPC]
88,9cdcf8bd45584c7097d1444920690609,labels,text,label,16270,16335,"The fine amount, if any, paid by the appellant...",[RPC]
89,9d3cd3d2675b4f78a936b4b3256b8862,labels,text,label,16335,16524,\n The office is directed to communicate this ...,[RPC]


In [ ]:
train_data = pd.DataFrame.from_dict(pd.json_normalize(train), orient='columns')
train_data.drop('annotations', axis=1, inplace=True)
train_data.drop('id', axis=1, inplace=True)
train_data.head()

,data.text,meta.group
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",Criminal
1,...,Tax
2,IN THE HIGH COURT OF KARNATAKA ...,Criminal
3,PETITIONER:\nRAGHUBAR MANDAL HARIHAR MANDAL\n\...,Tax
4,PETITIONER:\nP.K. BADIANI\n\n Vs.\n\nRESP...,Tax


In [ ]:
X = pd.concat([train_data, annotations], axis = 1)
X.head()

,data.text,meta.group,id,type,to_name,from_name,value.start,value.end,value.text,value.labels
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",Criminal,d7a902fe9c23417499a7ef782f9fbdeb,labels,text,label,0.0,116.0,"IN THE HIGH COURT OF KARNATAKA,\n ...",[PREAMBLE]
1,...,Tax,8d41599e98424d9480c25109556a7d14,labels,text,label,116.0,678.0,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,[PREAMBLE]
2,IN THE HIGH COURT OF KARNATAKA ...,Criminal,e501424117da40a7935c2d9f2fb2fe38,labels,text,label,678.0,964.0,This Criminal Appeal is filed under Section 37...,[PREAMBLE]
3,PETITIONER:\nRAGHUBAR MANDAL HARIHAR MANDAL\n\...,Tax,4825806388fe43d39f73354b10b5b32d,labels,text,label,964.0,1093.0,\n\n This appeal coming on for hearing t...,[PREAMBLE]
4,PETITIONER:\nP.K. BADIANI\n\n Vs.\n\nRESP...,Tax,d6893a25f82948f8be17fc9e876fb716,labels,text,label,1093.0,1180.0,\n Heard the learned Counsel for the app...,[NONE]


In [ ]:
labset = []
for lab in X["value.labels"]:
  if lab not in labset:
    labset.append(lab)
print(labset)

[['PREAMBLE'], ['NONE'], ['FAC'], ['ARG_RESPONDENT'], ['RLC'], ['ARG_PETITIONER'], ['ANALYSIS'], ['PRE_RELIED'], ['RATIO'], ['RPC'], nan]


In [ ]:
dev[0]['annotations'][0]['result']
annotations = pd.DataFrame.from_dict(pd.json_normalize(dev[0]['annotations'][0]['result']), orient='columns')
annotations

,id,type,to_name,from_name,value.start,value.end,value.text,value.labels
0,d7a902fe9c23417499a7ef782f9fbdeb,labels,text,label,0,116,"IN THE HIGH COURT OF KARNATAKA,\n ...",[PREAMBLE]
1,8d41599e98424d9480c25109556a7d14,labels,text,label,116,678,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,[PREAMBLE]
2,e501424117da40a7935c2d9f2fb2fe38,labels,text,label,678,964,This Criminal Appeal is filed under Section 37...,[PREAMBLE]
3,4825806388fe43d39f73354b10b5b32d,labels,text,label,964,1093,\n\n This appeal coming on for hearing t...,[PREAMBLE]
4,d6893a25f82948f8be17fc9e876fb716,labels,text,label,1093,1180,\n Heard the learned Counsel for the app...,[NONE]
...,...,...,...,...,...,...,...,...
86,375bebe6234540bf92865406f84bece0,labels,text,label,16196,16241,The judgment of the court below is set aside.,[RPC]
87,70cb6f740b1746259612a99bc2df8062,labels,text,label,16242,16269,The appellant is acquitted.,[RPC]
88,9cdcf8bd45584c7097d1444920690609,labels,text,label,16270,16335,"The fine amount, if any, paid by the appellant...",[RPC]
89,9d3cd3d2675b4f78a936b4b3256b8862,labels,text,label,16335,16524,\n The office is directed to communicate this ...,[RPC]


In [ ]:
dev_data = pd.DataFrame.from_dict(pd.json_normalize(dev), orient='columns')
dev_data.drop('annotations', axis=1, inplace=True)
dev_data.drop('id', axis=1, inplace=True)
dev_data.head()

,data.text,meta.group
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",Criminal
1,...,Tax
2,IN THE HIGH COURT OF KARNATAKA ...,Criminal
3,PETITIONER:\nRAGHUBAR MANDAL HARIHAR MANDAL\n\...,Tax
4,PETITIONER:\nP.K. BADIANI\n\n Vs.\n\nRESP...,Tax


In [ ]:
X_dev = pd.concat([dev_data, annotations], axis = 1)
X_dev.head()

,data.text,meta.group,id,type,to_name,from_name,value.start,value.end,value.text,value.labels
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",Criminal,d7a902fe9c23417499a7ef782f9fbdeb,labels,text,label,0.0,116.0,"IN THE HIGH COURT OF KARNATAKA,\n ...",[PREAMBLE]
1,...,Tax,8d41599e98424d9480c25109556a7d14,labels,text,label,116.0,678.0,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,[PREAMBLE]
2,IN THE HIGH COURT OF KARNATAKA ...,Criminal,e501424117da40a7935c2d9f2fb2fe38,labels,text,label,678.0,964.0,This Criminal Appeal is filed under Section 37...,[PREAMBLE]
3,PETITIONER:\nRAGHUBAR MANDAL HARIHAR MANDAL\n\...,Tax,4825806388fe43d39f73354b10b5b32d,labels,text,label,964.0,1093.0,\n\n This appeal coming on for hearing t...,[PREAMBLE]
4,PETITIONER:\nP.K. BADIANI\n\n Vs.\n\nRESP...,Tax,d6893a25f82948f8be17fc9e876fb716,labels,text,label,1093.0,1180.0,\n Heard the learned Counsel for the app...,[NONE]


In [ ]:
text = dev[0]['data']['text']
words = nltk.word_tokenize(text)

In [ ]:
pos_tags = nltk.pos_tag(words)

In [ ]:
pos_tags

In [ ]:
chunks = nltk.ne_chunk(pos_tags, binary=True)
#for chunk in chunks:
# print(chunk)

In [ ]:
entities = []
labels = []

for chunk in chunks:
  if hasattr(chunk, 'label'):
    entities.append(' '.join(c[0] for c in chunk))
    labels.append(chunk.label())

entites_labels = list(set(zip(entities, labels)))
entites_df = pd.DataFrame(entites_labels)
entites_df.columns = ["Entities", "labels"]
entites_df

,Entities,labels
0,Laxmibai,NE
1,Central Jail,NE
2,CIRCUIT,NE
3,Chhattisgarh,NE
4,Shri Ishwar Raj,NE
5,Taluk Chincholi,NE
6,GULBARGA,NE
7,Bidar District,NE
8,Bhagewati,NE
9,IPC,NE


# Training Dataset Cleanup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
df = pd.read_json("./drive/MyDrive/NER_TRAIN_JUDGEMENT.json")

In [ ]:
datas = {}
count = 0
for info in df['data']:
  datas.update({count:info.get('text')})
  count+=1

In [ ]:
metas = {}
count = 0
for info in df['meta']:
  metas.update({count:info.get('source').split("_")[0]})
  count+=1

In [ ]:
ids = {}
count = 0
for info in df['id']:
  ids.update({count:info})
  count+=1

In [ ]:
# load data using Python JSON module
with open('./drive/MyDrive/NER_TRAIN_JUDGEMENT.json','r') as f:
    judge = json.loads(f.read())
    
# Normalizing data
annotations = pd.json_normalize(judge, record_path =['annotations'])

In [ ]:
results = {}
count = 0
for entry in annotations['result']:
  if entry != []:
    for val in entry:
      results.update({str(count)+","+val.get('value').get('text'):val.get('value').get('labels')})
  count+=1

In [ ]:
com = pd.DataFrame()

i_col = range(len(ids.values()))
com['index'] = i_col

id_col = ids.values()
com['id'] = id_col

data_col = datas.values()
com['data'] = data_col

meta_col = metas.values()
com['meta'] = meta_col

com

,index,id,data,meta
0,0,90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
1,1,a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal
2,2,33cface31108441080976035ee04d07e,"\n5.2 CW3 Mr Vijay Mishra , Deputy Manager, H...",financial
3,3,08debe900b51464e9f26b5d1eecae170,You are hereby asked not to carry out any cons...,civil
4,4,428070222cd940eba78b233829eb28b8,The pillion rider T.V. Satyanarayana Murthy al...,motorvehicles
...,...,...,...,...
9430,9430,5cb430a6d0e14e57a968f0c43d219682,It is prayed in the application that the suit ...,civil
9431,9431,6e9147c9e6344865a8b095fb7f43f7c2,"In the first instance, Mr.A.D.Desai contended ...",tax
9432,9432,657ef5df817d4e95a36d702b4e41ce7d,"Sikri, J. (as he then was), speaking for the m...",constitution
9433,9433,9ff4b03ee09f49a6814b9550c145ea1e,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,industrial&labour


In [ ]:
train = pd.DataFrame()

texts = []
labs = []
cids = []
for entry in results:
  labs.append(results.get(entry))
  cid = entry.split(",")[0]
  text = entry.split(",")[1]
  texts.append(text)
  cids.append(cid)
  count+=1

train['text'] = texts
train['label'] = labs
train['case id'] = cids

train

,text,label,case id
0,Hongkong Bank,[ORG],0
1,Rahul & Co.,[ORG],0
2,Agya,[OTHER_PERSON],1
3,Kaur,[OTHER_PERSON],1
4,Tarlochan Singh,[OTHER_PERSON],1
...,...,...,...
16992,RAJENDRA MAHAJAN,[JUDGE],9433
16993,M.Cr.C. No. 8763/2009,[CASE_NUMBER],9433
16994,04/02/2015,[DATE],9433
16995,Wakeel Khan,[OTHER_PERSON],9433


In [ ]:
full = []
fmeta = []
fid = []
for idx, entity in train.iterrows():
  num = int(entity["case id"])
  full.append(com.at[num,"data"])
  fmeta.append(com.at[num,"meta"])
  fid.append(com.at[num,"id"])

train["case"] = full
train["meta"] = fmeta
train["case id"] = fid

train

,text,label,case id,case,meta
0,Hongkong Bank,[ORG],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
1,Rahul & Co.,[ORG],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
2,Agya,[OTHER_PERSON],a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal
3,Kaur,[OTHER_PERSON],a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal
4,Tarlochan Singh,[OTHER_PERSON],a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal
...,...,...,...,...,...
16992,RAJENDRA MAHAJAN,[JUDGE],9ff4b03ee09f49a6814b9550c145ea1e,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,industrial&labour
16993,M.Cr.C. No. 8763/2009,[CASE_NUMBER],9ff4b03ee09f49a6814b9550c145ea1e,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,industrial&labour
16994,04/02/2015,[DATE],9ff4b03ee09f49a6814b9550c145ea1e,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,industrial&labour
16995,Wakeel Khan,[OTHER_PERSON],9ff4b03ee09f49a6814b9550c145ea1e,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,industrial&labour


In [ ]:
df2 = pd.read_json("./drive/MyDrive/NER_TRAIN_PREAMBLE.json")

In [ ]:
datas2 = {}
count = 0
for info in df2['data']:
  datas2.update({count:info.get('text')})
  count+=1

In [ ]:
metas2 = {}
count = 0
for info in df2['meta']:
  metas2.update({count:info.get('source').split("_")[0]})
  count+=1

In [ ]:
ids2 = {}
count = 0
for info in df2['id']:
  ids2.update({count:info})
  count+=1

In [ ]:
# load data using Python JSON module
with open('./drive/MyDrive/NER_TRAIN_PREAMBLE.json','r') as f:
  pre2 = json.loads(f.read())
    
# Normalizing data
annotations2 = pd.json_normalize(pre2, record_path =['annotations'])

In [ ]:
results2 = {}
count = 0
for entry in annotations2['result']:
  if entry != []:
    for val in entry:
      results2.update({str(count)+","+val.get('value').get('text'):val.get('value').get('labels')})
  count+=1

In [ ]:
com2 = pd.DataFrame()

i_col2 = range(len(ids2.values()))
com2['index'] = i_col2

id_col2 = ids2.values()
com2['id'] = id_col2

data_col2 = datas2.values()
com2['data'] = data_col2

meta_col2 = metas2.values()
com2['meta'] = meta_col2

com2

,index,id,data,meta
0,0,d79fb7f965a74e418212458285c7c213,In The High Court Of Kerala At Ernakulam\n\nCr...,criminal
1,1,10d82191b30e4e9d9b0ab91c34606631,In The Court Of Shri Lokesh Kumar Sharma\n ...,constitution
2,2,b150ace41ceb4acf87d12f75b4748ffc,Before The Madurai Bench Of Madras High Court\...,civil
3,3,278f93b03a484705b3d96a72aa09be26,Before The Madurai Bench Of Madras High Court\...,tax
4,4,ceab6a1b200242208a4d6beb042d8743,1 ...,criminal
...,...,...,...,...
1555,1555,2b617380eac44e85a80d53e9c067360b,In The High Court Of Judicature At Madras ...,motorvehicles
1556,1556,799133f87be04599a0b5367a4a1a23bf,Petitioner:\nKhudiram Das\n\n\tVs.\n\nResponde...,criminal
1557,1557,d21055fa778e4579a5711c6ffb1b9560,In The High Court Of Judicature At Patna\n\n ...,constitution
1558,1558,307722b0639841c9b31418fbc84d7a52,Petitioner:\nThe Automobile Transport(Rajastha...,tax


In [ ]:
train2 = pd.DataFrame()

texts2 = []
labs2 = []
cids2 = []
for entry in results2:
  labs2.append(results2.get(entry))
  cid = entry.split(",")[0]
  text = entry.split(",")[1]
  texts2.append(text)
  cids2.append(cid)
  count+=1

train2['text'] = texts2
train2['label'] = labs2
train2['case id'] = cids2

train2

,text,label,case id
0,High Court Of Kerala At Ernakulam,[COURT],0
1,T.R.Ajayan,[PETITIONER],0
2,M.Ravindran,[RESPONDENT],0
3,Nirmala Dinesh,[RESPONDENT],0
4,A.Kumar,[LAWYER],0
...,...,...,...
11257,V. Nath,[JUDGE],1557
11258,Automobile Transport(Rajasthan) Ltd.,[PETITIONER],1558
11259,State Of Rajasthan,[RESPONDENT],1558
11260,East India COMMERClAL Co.,[PETITIONER],1559


In [ ]:
full2 = []
fmeta2 = []
fid2 = []
for idx, entity in train2.iterrows():
  num = int(entity["case id"])
  full2.append(com.at[num,"data"])
  fmeta2.append(com.at[num,"meta"])
  fid2.append(com.at[num,"id"])

train2["case"] = full2
train2["meta"] = fmeta2
train2["case id"] = fid2

train2

,text,label,case id,case,meta
0,High Court Of Kerala At Ernakulam,[COURT],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
1,T.R.Ajayan,[PETITIONER],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
2,M.Ravindran,[RESPONDENT],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
3,Nirmala Dinesh,[RESPONDENT],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
4,A.Kumar,[LAWYER],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
...,...,...,...,...,...
11257,V. Nath,[JUDGE],dd0bd3b3088541909bf3c88481254dd4,Lord Dunedin expressed the view that there mig...,industrial&labour
11258,Automobile Transport(Rajasthan) Ltd.,[PETITIONER],91efcd1b8a46467c8e772f736ae4b41d,Ugamchand D.W. 14 also corroborates Inderchand...,civil
11259,State Of Rajasthan,[RESPONDENT],91efcd1b8a46467c8e772f736ae4b41d,Ugamchand D.W. 14 also corroborates Inderchand...,civil
11260,East India COMMERClAL Co.,[PETITIONER],71671326f0f64d8da7f5f4c5a08ce156,It is pertinent to mention here that from the ...,criminal


In [ ]:
cat_col = []
for i in range(len(train)):
  cat_col.append("JUDGEMENT")
train["category"] = cat_col

cat_col2 = []
for i in range(len(train2)):
  cat_col2.append("PREAMBLE")
train2["category"] = cat_col2

In [ ]:
train_data = pd.concat([train, train2], ignore_index=True)
train_data[500:530]

,text,label,case id,case,meta,category
500,Hema Kulkarni,[OTHER_PERSON],53320c93fd844392b902271820c6bb96,"Smt.Hema Kulkarni, the learned counsel appears...",criminal,JUDGEMENT
501,Criminal Appeal No. 372/2008,[CASE_NUMBER],53320c93fd844392b902271820c6bb96,"Smt.Hema Kulkarni, the learned counsel appears...",criminal,JUDGEMENT
502,s. 80E(1),[PROVISION],f0a1291a1b7e4d8e84f082fd4503acd2,In other words the High Court upheld the view ...,financial,JUDGEMENT
503,Tilakchand,[WITNESS],2bc621f6606546c88aacdfd07583e37b,"P.W. 35 the wood stall keeper Tilakchand, who ...",financial,JUDGEMENT
504,Veeranna Satnoor,[WITNESS],85eed4936ef74f2db6f6700457b900c8,"However, the learned member of the Tribunal di...",motorvehicles,JUDGEMENT
505,Court of Judicial Magistrate,[COURT],d56883a9bf604149a9156d4a1e6918c5,"After when the investigation was over, the pol...",criminal,JUDGEMENT
506,Chimpirayya,[OTHER_PERSON],1c8fe768d75c4fb4a690ac8b57114fce,she had knowledge of it only after the death o...,land&property,JUDGEMENT
507,11.10.2002,[DATE],38468f529a08419b961cf21333592903,The Court below has come to the conclusion tha...,civil,JUDGEMENT
508,,[DATE],38468f529a08419b961cf21333592903,The Court below has come to the conclusion tha...,civil,JUDGEMENT
509,15.11.2002,[DATE],38468f529a08419b961cf21333592903,The Court below has come to the conclusion tha...,civil,JUDGEMENT


# My Failed Attempt at Data Manip/IOB Labeling :(

In [ ]:
ner_labs = []
for lab in train_data['label']:
  if not lab in ner_labs:
    ner_labs.append(lab)

print(ner_labs)

TypeError: ignored

In [ ]:
train_tok = []
prev = ""

for case in train_data["case"]:
  if not case == prev:
    case_tok = case.split(" ")
    for word in case_tok:
      train_tok.append(word)
    train_tok.append("<END>")
    prev = case

print(len(train_tok))

321822


In [ ]:
import re

train_lab = []
train_ent = []

ents = train_data["text"]
labs = train_data["label"]
idx = 0
add = 0
for i in range(len(train_tok)):
  ent_list = ents[idx].split(" ")
  ln = len(ent_list)
  match = True
  if i + add + ln < len(train_tok):
    for n in range(ln):
      if re.sub('\W', '', ent_list[n]) not in re.sub('\W', '', train_tok[i + add + n]):
        match = False

    if match == True:
      # print(train_tok[i + add])
      check = ""
      if ent_list[n] == "9.12.2002":
          check = ent_list[n]
      for n in range(ln):
        if check == "9.12.2002":
          print(train_tok[i + add + n])
          print(labs[idx])
        train_ent.append(train_tok[i + add + n])
        train_lab.append(labs[idx])
        if check == "9.12.2002":
          print("HELLO")
          print(labs[idx])
      add += ln - 1
      if idx < len(train_data):
        idx += 1
    else:
      train_ent.append(train_tok[i + add])
      train_lab.append("O")

print(len(train_tok))
print(len(train_ent))
print(len(train_lab))

9.12.2002,
['DATE']
HELLO
['DATE']
321822
321818
321818


In [ ]:
for i in range(len(train_ent)):
  print(str(train_ent[i]) + ", " + str(train_lab[i]))

Streaming output truncated to the last 5000 lines.
The, O
Investigating, O
Officer,, O
Himatgiri, ['WITNESS']
Gosai,, ['WITNESS']
was, O
examined, O
and, O
police, O
statement, O
of, O
Laljibhai, ['OTHER_PERSON']
was, O
duly, O
proved, O
through, O
his, O
evidence., O
<END>, O


21., O
Section, ['PROVISION']
173(8), ['PROVISION']
is, O
a, O
new, O
provision, O
introduced, O
by, O
the, O
Code, ['STATUTE']
of, ['STATUTE']
Criminal, ['STATUTE']
Procedure,, ['STATUTE']
1973, O
(hereinafter, O
referred, O
to, O
as, O
„Code‟),, O
which, O
was, O
not, O
there, O
in, O
the, O
earlier, O
Code., O
<END>, O
The, O
petitioner, O
was, O
also, O
examined, O
on, O
28.4.2014, ['DATE']
in, O
terms, O
of, O
the, O
judgment, O
of, O
the, O
Hon'ble, O
High, O
Court, O
on, O
11.01.2013, ['DATE']
in, O
MACA, ['CASE_NUMBER']
NO., ['CASE_NUMBER']
792/2006, ['CASE_NUMBER']
titled, O
as, O
Oriental, ['PRECEDENT']
Insurance, ['PRECEDENT']
Co., ['PRECEDENT']
Ltd, ['PRECEDENT']
v., ['PRECEDENT']
Ranjit, ['PRECEDEN

KeyboardInterrupt: ignored

# Spacy Data Prep

In [8]:
# combine json files (NER_TRAIN_JUDGEMENT_mod.json and NER_TRAIN_PREAMBLE_mod.json) into one

In [9]:
import logging
import argparse
import sys
import os
import json
import pickle
import requests
from spacy.tokenizer import Tokenizer

train_data = []

nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

def main(input_file, output_file):
    try:
        training_data = []
        jsons=[]
        f = open(input_file, 'r')
        jsons = json.load(f)

        for item in jsons:
            # response = requests.get()
            # print(item)
            case = item["data"]
            text = case["text"]
            entities = []
            for annotation in item["annotations"]:
                result = annotation["result"]
                for val in result:
                    # print(value)
                    value = val["value"]
                    ent = value["text"]
                    labels = value["labels"]
                    if not isinstance(labels, list):
                        labels = [labels]    

                    tokent = tokenizer(ent)
                    toktext = tokenizer(text)
                    start = 0
                    end = 0
                    for ind in range(len(toktext) - len(tokent)):
                        for tok in tokent:
                            match = True
                            if tok != toktext[ind]:
                                match = False
                        if match == True:
                            start = ind
                            end = ind + len(tokent)


                    for label in labels:
                        entities.append((start, end, label))


            training_data.append((text, {"entities" : entities}))
            train_data.append((text, {"entities" : entities}))

        print(train_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None


main("./NER_TRAIN_JUDGEMENT_mod.json", "./out")

[("\n\n(7) On specific query by the Bench about an entry of Rs. 1,31,37,500 on deposit side of Hongkong Bank account of which a photo copy is appearing at p. 40 of assessee's paper book, learned authorised representative submitted that it was related to loan from broker, Rahul & Co. on the basis of his submission a necessary mark is put by us on that photo copy.", {'entities': [(0, 0, 'ORG'), (0, 0, 'ORG')]}), ('He was also asked whether Agya <span class="hidden_text" id="span_5"> CRA No.326-DB of 1998 6</span> Kaur, mother-in-law of the deceased lived separately from Tarlochan Singh.', {'entities': [(0, 0, 'OTHER_PERSON'), (0, 0, 'OTHER_PERSON'), (0, 0, 'OTHER_PERSON')]}), (' \n5.2 CW3 Mr Vijay Mishra , Deputy Manager, HDFC Bank, Noida, UP has deposed that complainant had a current account with HDFC Bank in the year 2004\xad2005.', {'entities': [(0, 0, 'WITNESS'), (0, 0, 'ORG'), (0, 0, 'GPE'), (0, 0, 'ORG')]}), ('You are hereby asked not to carry out any construction work of the said 

In [10]:
for case in train_data:
    ents = case[1].get('entities')
    for ent in ents:
        if ent[0] == ent[1]:
            ents.remove(ent)

# Spacy Model

In [ ]:
# !python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [13]:
# !python -m spacy init config - --lang en --pipeline ner --optimize accuracy | python -m spacy train - --paths.train ./train.spacy --paths.dev ./dev.spacy

ℹ No output directory provided
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-12-13 18:23:04,342] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-13 18:23:04,352] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-12-13 18:23:04,356] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-13 18:23:06,556] [INFO] Added vectors: en_core_web_lg
INFO:spacy:Added vectors: en_core_web_lg
tcmalloc: large alloc 1643593728 bytes == 0x47172000 @  0x7f788c92a2a4 0x7f7881727e09 0x7f7881726cdf 0x7f7881723675 0x7f7881723e2e 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x7f774ae77d38 0x7f774ae7a6ae 0x7f774ae82187 0x7f774ae8559c 0x5d8d8c 0x55dc1e 0x5d8868 0x4990ca 0x7f774ae77d38 0x7f774ae7a6ae 0x7f774ae81a2b 0x5d80be 0x5d8d8c 0x55ea20 0x55d078 0x5d8941 0x49abe4 0x4fe253 0x49abe4 0x55d078 0x5d8941
tcmalloc: large alloc 16625090

In [11]:
import en_core_web_lg
import configparser

nlp.select_pipes(disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])
configparse = configparser.ConfigParser()
# nlp = spacy.blank('en_core_web_trf')  # create blank Language class

if 'ner' not in nlp.pipe_names:
    nlp.add_pipe('ner', config = configparse.read("./config.cfg"))
ner = nlp.get_pipe('ner')

In [12]:
LABEL = ["[\"COURT\"]", "[\"PETITIONER\"]", "[\"DATE\"]", "[\"PLAINTIFF\"]", "[\"RESPONDENT\"]", "[\"JUDGE\"]", "[\"LAWYER\"]", "[\"ORG\"]", "[\"GPE\"]", "[\"STATUTE\"]", "[\"PROVISION\"]", "[\"CASE_NUMBER\"]", "[\"PRECEDENT\"]", "[\"WITNESS\"]", "[\"OTHER_PERSON\"]"]
for i in LABEL:
    ner.add_label(i)
# Inititalizing optimizer
optimizer = nlp.get_pipe("ner").create_optimizer()

In [17]:
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import Language

matcher = Matcher(nlp.vocab)

# ruler = nlp.add_pipe("entity_ruler")
# define the pattern
pattern =[
 {"label": "[\"DATE\"]", "pattern": {"REGEX": "\d\d\.\d.\d\d\d\d|\d\d\.\d\d.\d\d\d\d"}}
]
date_pattern = [{"TEXT": {"REGEX": "\d\d\.\d.\d\d\d\d|\d\d\.\d\d.\d\d\d\d"}}]
# add the pattern to the matcher object

matcher.add("[\"DATE\"]", [date_pattern])

@Language.factory("matchers")
class Matchers:
    def __init__(self, nlp: Language, name: str = "my_component") -> None:
        self.name = name
        self.data = []
    def __call__(self, doc):
        # Use the matchers on the doc and retreive the informations of interest
        spans = []
        filter_names = []
        matches = matcher(doc)
        doc.cats["n_matches"] = len(matches)
        for match_id, start, end in matches:
            spans.append(doc[start:end])
            filter_names.append(nlp.vocab.strings[match_id])
        # Append the span matched
        doc.cats["spans"] = spans
        # Append the filter that matched
        doc.cats["filter_name"] = filter_names
        return doc
    def add(self, data):
        # Add something to the component's data
        self.data.append(data)

nlp.add_pipe("matchers")
# nlp.to_disk("trained_ner")

In [ ]:
from spacy.util import minibatch, compounding
from spacy.training.example import Example
import random

n_iter = 20
for itn in range(n_iter):
    print("iter: " + str(itn) + "/" + str(n_iter))
    random.shuffle(train_data)
    losses = {}
    batches = minibatch(train_data, size=compounding(4., 32., 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
            nlp.update(example, drop=0.5, losses=losses)
            # for text, annotations in batch:
              # Updating the weights
              # doc = nlp.make_doc(text)
              # example = Example.from_dict(doc, annotations)
              # nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)
            # print('Losses', losses)
            # nlp.update(texts, annotations, sgd=optimizer, 
                       # drop=0.35, losses=losses)
        # print('Losses', losses)

iter: 0/20


# Export Model

In [23]:
from pathlib import Path

# Save model 
output_dir = Path("./spacy_model/")
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = "spacy_v6"  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to spacy_model


In [24]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
test_text = 'In view of these fact, it can he safely held that for the area of Jaipur City (municipal limits) there were two Special Judges on 29.4.1968, one was Sessions Judge Jaipur City, who by virtue of his Office was appointed to be Special Judge for that area by virtue of-Notified ion dated 26.2.1968.'
doc = nlp2(test_text)
print("Entities in '%s'" % test_text)

for ent in doc.ents:
    print(ent.label_, ent.text)

Loading from spacy_model
Entities in 'In view of these fact, it can he safely held that for the area of Jaipur City (municipal limits) there were two Special Judges on 29.4.1968, one was Sessions Judge Jaipur City, who by virtue of his Office was appointed to be Special Judge for that area by virtue of-Notified ion dated 26.2.1968.'


/usr/local/lib/python3.8/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))


In [25]:
!zip -r ./spacy_model.zip ./spacy_model/

  adding: spacy_model/ (stored 0%)
  adding: spacy_model/ner/ (stored 0%)
  adding: spacy_model/ner/model (deflated 8%)
  adding: spacy_model/ner/moves (deflated 82%)
  adding: spacy_model/ner/cfg (deflated 33%)
  adding: spacy_model/config.cfg (deflated 73%)
  adding: spacy_model/attribute_ruler/ (stored 0%)
  adding: spacy_model/attribute_ruler/patterns (deflated 84%)
  adding: spacy_model/parser/ (stored 0%)
  adding: spacy_model/parser/model (deflated 7%)
  adding: spacy_model/parser/moves (deflated 55%)
  adding: spacy_model/parser/cfg (deflated 32%)
  adding: spacy_model/senter/ (stored 0%)
  adding: spacy_model/senter/model (deflated 9%)
  adding: spacy_model/senter/cfg (stored 0%)
  adding: spacy_model/lemmatizer/ (stored 0%)
  adding: spacy_model/lemmatizer/lookups/ (stored 0%)
  adding: spacy_model/lemmatizer/lookups/lookups.bin (deflated 56%)
  adding: spacy_model/tokenizer (deflated 80%)
  adding: spacy_model/vocab/ (stored 0%)
  adding: spacy_model/vocab/vectors.cfg (store

In [26]:
from google.colab import files
files.download("./spacy_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Development Testing

In [14]:
!unzip spacy_model.zip -d spacy_model

Archive:  spacy_model.zip
   creating: spacy_model/spacy_model/
   creating: spacy_model/spacy_model/ner/
  inflating: spacy_model/spacy_model/ner/model  
  inflating: spacy_model/spacy_model/ner/moves  
  inflating: spacy_model/spacy_model/ner/cfg  
  inflating: spacy_model/spacy_model/config.cfg  
   creating: spacy_model/spacy_model/attribute_ruler/
  inflating: spacy_model/spacy_model/attribute_ruler/patterns  
   creating: spacy_model/spacy_model/parser/
  inflating: spacy_model/spacy_model/parser/model  
  inflating: spacy_model/spacy_model/parser/moves  
  inflating: spacy_model/spacy_model/parser/cfg  
   creating: spacy_model/spacy_model/senter/
  inflating: spacy_model/spacy_model/senter/model  
 extracting: spacy_model/spacy_model/senter/cfg  
   creating: spacy_model/spacy_model/lemmatizer/
   creating: spacy_model/spacy_model/lemmatizer/lookups/
  inflating: spacy_model/spacy_model/lemmatizer/lookups/lookups.bin  
  inflating: spacy_model/spacy_model/tokenizer  
   creatin

In [19]:
from pathlib import Path

dev_data = []
jsons=[]
f = open("./NER_DEV_JUDGEMENT_mod.json", 'r')
dev_j = json.load(f)

model_dir = Path("./spacy_model/spacy_model/")

print("Loading from", model_dir)
nlp3 = spacy.load(model_dir)

for item in dev_j:
    case = item["data"]
    text = case["text"]
    dev_data.append("text")

    dev_ents = []
    for annotation in item["annotations"]:
        result = annotation["result"]
        for val in result:
            # print(value)
            value = val["value"]
            ent = value["text"]
            labels = value["labels"]
            if not isinstance(labels, list):
                labels = [labels]    

            tokent = tokenizer(ent)
            toktext = tokenizer(text)
            start = 0
            end = 0
            for ind in range(len(toktext) - len(tokent)):
                for tok in tokent:
                    match = True
                    if tok != toktext[ind]:
                        match = False
                if match == True:
                    start = ind
                    end = ind + len(tokent)


            for label in labels:
                dev_ents.append((start, end, label))


pred_ents = []

for sent in dev_data:
    doc = nlp3(sent)
    doc_ents = []
    for ent in doc.ents:
        doc_ents.append((ent.label_, ent.text))
    pred_ents.append(doc_ents)

print("Pred: " + str(pred_ents[0]))
print("Real: " + str(dev_ents[0]))

Loading from spacy_model/spacy_model


/usr/local/lib/python3.8/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.8/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.8/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.8/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.8/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any pattern

Pred: []


IndexError: ignored

In [ ]:
def precision(
    predicted_labels: List[torch.Tensor],
    true_labels: List[torch.Tensor],
    outside_tag_idx: int
):
    """
    Precision is True Positives / All Positives Predictions
    """
    TP = torch.tensor([0])
    denom = torch.tensor([0])
    for pred, true in zip(predicted_labels, true_labels):
        TP += sum((pred == true)[pred != outside_tag_idx])
        denom += sum(pred != outside_tag_idx)

    # Avoid division by 0
    denom = torch.tensor(1) if denom == 0 else denom
    return TP / denom


def recall(
    predicted_labels: List[torch.Tensor],
    true_labels: List[torch.Tensor],
    outside_tag_idx: int
):
    """
    Recall is True Positives / All Positive Labels
    """
    TP = torch.tensor([0])
    denom = torch.tensor([0])
    for pred, true in zip(predicted_labels, true_labels):
        TP += sum((pred == true)[true != outside_tag_idx])
        denom += sum(true != outside_tag_idx)

    # Avoid division by 0
    denom = torch.tensor(1) if denom == 0 else denom
    return TP / denom


def f1_score(predicted_labels, true_labels, outside_tag_idx):
    """
    F1 score is the harmonic mean of precision and recall
    """
    P = precision(predicted_labels, true_labels, outside_tag_idx)
    R = recall(predicted_labels, true_labels, outside_tag_idx)
    return 2*P*R/(P+R)

In [ ]:
dev_f1 = f1_score(predict(pred_ents, dev_ents, outside_tag_idx='O')

print(f"Dev F1 {dev_f1}")

# CRF Model Setup (Backup)

In [ ]:
poss = [0]

# TODO: Update this function to add more features
#      You can check crf.py for how they are encoded, if interested.
def pos_feature(word):
    if word not in poss:
        poss.append(word)
    return poss.index(word)

def make_features(text: List[str], dataset) -> List[List[int]]:
    """Turn a text into a feature vector.

    Args:
        text (List[str]): List of tokens.

    Returns:
        List[List[int]]: List of feature Lists.
    """
    prev = 0
    feature_lists = []
    pos_list = nltk.pos_tag(text)
    for i, token in enumerate(text):
        feats = []
        pos = pos_list[i][1]
        # We add a feature for each unigram.
        feats.append(f"word={token}")
        # TODO: Add more features here
        feats.append(pos_feature(pos))
        if prev == 0:
            feats.append(0)
        else:
            prev_pos = pos_list[i-1][1]
            feats.append(pos_feature(prev_pos))

        # We append each feature to a List for the token.
        feature_lists.append(feats)
        prev = token

    return feature_lists

In [ ]:
def featurize(sents: List[List[str]]) -> List[List[List[str]]]:
    """Turn the sentences into feature Lists.
    
    Eg.: For an input of 1 sentence:
         [[['I','am','a','student','at','CU','Boulder']]]
        Return list of features for every token for every sentence like:
        [[
         ['word=I',  'prev_word=<S>','pos=PRON',...],
         ['word=an', 'prev_word=I'  , 'pos=VB' ,...],
         [...]
        ]]

    Args:
        sents (List[List[str]]): A List of sentences, which are Lists of tokens.

    Returns:
        List[List[List[str]]]: A List of sentences, which are Lists of feature Lists
    """
    feats = []
    for sent in sents:
        # sent_tags = get_pos(pos_tagger, [sent])[0]
        # Gets a List of Lists of feature strings

        # Note: I tried so hard to use the POS tagger here and send
        # in the fully tagged sentence as a parameter to make_features
        # but my computer already took 5 hours to run the code just 
        # with the individual words POS tagged so when I tried to tag
        # the whole sentence it just ran for 2 hours without any output :/
        feats.append(make_features(sent))
        
        # TO DO: Get pos tags
        

    return feats

In [ ]:
# For feature encodings. We have a special UNKNOWN feature
UNK_SYMBOL = "<UNK>"
# For padding our variable length batches
PAD_SYMBOL = "<PAD>"


def load_data(
    filepath: str,
) -> Tuple[List[str], List[str]]:
    """Load the training data, producing a List of sentences, each comprising
    a List of tokens, and a List of their corresponding NER tags

    Args:
        filepath (str): Path to the training file

    Returns:
       Tuple[List[str], List[str]]: The tokens and tags
    """
    token_sents = []
    tag_sents = []
    token_sent = []
    tag_sent = []
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            # Newline indicates a new sentence.
            if not line.rstrip():
                token_sents.append(token_sent)
                token_sent = []
                tag_sents.append(tag_sent)
                tag_sent = []
            else:
                token, tag = line.rstrip().split("\t")
                token_sent.append(token)
                tag_sent.append(tag)

    return token_sents, tag_sents


def make_labels2i(labels_filepath: str):
    # Initialize labels2i with the PAD_SYMBOL.
    labels2i = {PAD_SYMBOL: 0}
    i = 1
    with open(labels_filepath, "r", encoding="utf-8") as f:
        for line in f:
            if line.rstrip():
                labels2i[line.rstrip()] = i
                i += 1

    
    return labels2i

In [ ]:
def make_features_dict(all_features: Set) -> Dict:
    # Add 2 to each feature index to reserve 0 for PAD and 1 for UNK
    features_dict = {f: i+2 for i, f in enumerate(all_features)}
    features_dict[PAD_SYMBOL] = 0
    features_dict[UNK_SYMBOL] = 1
    print(f"Found {len(features_dict)} features")

    return features_dict


def build_features_set(train_features: List[List[List[str]]]) -> Set:
    """Build a set of all possible features. We limit this to only those in train.

    Args:
        train_features (List[List[List[str]]]): A List representing sentences.

    Returns:
        Set: The set of all unique features
    """
    all_features = set()

    print("Building features set!")
    for features_sent in tqdm(train_features):
        for features in features_sent:
            for f in features:
                all_features.add(f)

    return all_features


def encode_features(
    feature_sents: List[List[List[str]]], features_dict: Dict[str, int]
) -> List[torch.LongTensor]:
    """Encode the features with features_dict.

    Args:
        feature_sents (List[List[List[str]]]): The List of sentences, of feature Lists.
        features_dict (Dict[str, int]): The encoding dict.

    Returns:
        List[torch.LongTensor]: A List of Tensors with encoded features
    """
    encoded_features = []
    
    for feature_sent in feature_sents:
        encoded = [
            encode_token_features(token_features, features_dict) for token_features in feature_sent
        ]
        encoded_features.append(torch.stack(encoded))

    return encoded_features


def encode_labels(tag_sents: List[List[str]], labels2i: Dict[str, int]) -> List[torch.LongTensor]:
    """Encode the labels using label2i. This results in one Tensor for each sentence.

    Args:
        tag_sents (List[List[str]]): The tag for each token in each sentence.
        labels2i (Dict[str, int]): The encoding dict for labels.

    Returns:
        List[torch.LongTensor]: Encoded labels.
    """
    encoded_labels = []
    for labels in tag_sents:
        encoded_labels.append(torch.LongTensor([
            labels2i[l] for l in labels
        ]))

    return encoded_labels


def encode_token_features(features: List[str], features_dict: Dict[str, int]) -> torch.Tensor:
    """Turn the features into a Tensor of integers.
    Note that we let nn.Embedding handle sparse layer, so we can pass a dense tensor of
    indices, rather than an indicator tensor.

    Args:
        features (List[str]): The string features to encode.
        features_dict (Dict[str, int]): The encoding of features to indices.

    Returns:
        torch.Tensor: The encoded features.
    """
    return torch.LongTensor([
        features_dict.get(feat, features_dict[UNK_SYMBOL]) for feat in features
    ])

In [ ]:
def predict(model: torch.nn.Module, feature_sents: List[List[int]]) -> List[torch.Tensor]:
    """Make predictions with the input model. Return a List of tensors.

    Args:
        model (torch.nn.Module): The trained model.
        feature_sents (List[List[int]]): The feature Lists for each sentence.

    Returns:
        List[torch.Tensor]: A List of predicted integers.
    """
    out = []
    for features in feature_sents:
        # Dummy batch dimension
        features = features.unsqueeze(0)
        # -> List[List[int]]
        preds = model.decode(features)
        preds = [torch.tensor(p) for p in preds]
        out.extend(preds)

    return out

# CRF Model (Backup)

In [ ]:
class NERTagger(torch.nn.Module):
    """NER tagger in pytorch, 
    relying on (pytorch-crf)[https://pytorch-crf.readthedocs.io/en/stable/]
    """
    def __init__(self, features_dim: int, num_tags: int):
        """
        Args:
            num_tags (int): The number of NER tags.
            features_dim (int): Dimension of each feature vector.
                This should correspond to the number of possible features.
        """
        super().__init__()
        self.num_tags = num_tags
        self.features_dim = features_dim
        # The matrix for computing emission probabilities before
        # incorporating the label sequence in the CRFDecoder.
        # NOTE: We use nn.Embedding for its sparse implementation,
        #   but this is not actually getting embeddings in the general sense
        #   of learning representations. If you are interested in this detail, please ask!
        self.emissions_scorer = torch.nn.Embedding(features_dim, num_tags)
        # Initialize the CRF using pytorch-crf. This adds the structured prediction function 
        # on top of our scorer. Note that the scorer works essentially like a logistic regression classifier
        # But we add the CRF on top.
        self.crf_decoder = CRFDecoder(self.num_tags, batch_first=True)

    def forward(
        self, input_seq: torch.Tensor, tags: torch.Tensor, mask: torch.Tensor
    ) -> torch.Tensor:
        """Compute the log likelihood of the gold tags. This assumes
        we are in training mode.

        Args:
            input_seq (torch.Tensor): sequence_length x batch_size x num_features tensor
                of the featurized input sentence.
            tags (torch.Tensor): sequence_length x batch_size tensor of the tags.
            mask (torch.Tensor): THe mask marking which tokens are dummy pads.

        Returns:
             torch.Tensor: A batch_size x 1 tensor of the (float) sum of log_likelihoods
                for each tag.
        """
        # Get the probability of each label for each token.
        emissions = self.make_emissions(input_seq)
        # Compute the log likelihood of the gold tags 
        # given the input. We get y_i and y_{i-1} from the gold tags.
        return self.crf_decoder(emissions, tags, mask=mask, reduction="mean")

    def decode(self, input_seq: torch.Tensor) -> List[int]:
        emissions = self.make_emissions(input_seq)
        # Decode the argmax sequence of labels with viterbi
        return self.crf_decoder.decode(emissions)

    def make_emissions(self, input_seq: torch.Tensor) -> torch.Tensor:
        """Compute a probability distribution over the tags for each
        input word.

        Args:
            input_seq (torch.Tensor): batch_size x sequence_length x num_features tensor
                of the featurized input sentence.
        """
        # -> batch_size x seq_len x num_tags
        # This gives a distribution over the tags for each token
        # in the input sequence
        # We sum over the 2nd to last dim (#features) because we use embeddings as a hack
        # to get efficient sparse implementation.
        # Embedding each feature and summing is equivalent to the matrix-multiply
        # of a single layer.
        return torch.sum(self.emissions_scorer(input_seq), dim=-2)

In [ ]:
import random
from tqdm.autonotebook import tqdm

# TODO: Implement the training loop
# HINT: Build upon what we gave you for HW2.
# See cell below for how we call this training loop.

def training_loop(
    num_epochs,
    batch_size,
    train_features,
    train_labels,
    dev_features,
    dev_labels,
    optimizer,
    model,
    labels2i,
    pad_feature_idx
):
    # TODO: Zip the train features and labels
    samples = list(zip(train_features, train_labels))
    # TODO: Randomize them, while keeping them paired.
    random.shuffle(samples)
    # TODO: Build batches
    batches = []
    for i in range(0, len(samples), batch_size):
        batches.append(samples[i:i+batch_size])
    print("Training...")
    for i in range(num_epochs):
        losses = []
        for batch in tqdm(batches):
            # Here we get the features and labels, pad them,
            # and build a mask so that our model ignores PADs
            # We have abstracted the padding from you for simplicity, 
            # but please reach out if you'd like learn more.
            features, labels = zip(*batch)
            features = pad_features(features, pad_feature_idx)
            features = torch.stack(features)
            # Pad the label sequences to all be the same size, so we
            # can form a proper matrix.
            labels = pad_labels(labels, labels2i[PAD_SYMBOL])
            labels = torch.stack(labels)
            mask = (labels != labels2i[PAD_SYMBOL])
            # TODO: Empty the dynamic computation graph
            optimizer.zero_grad()
            # TODO: Run the model. Since we use the pytorch-crf model,
            # our forward function returns the positive log-likelihood already.
            # We want the negative log-likelihood. See crf.py forward method in NERTagger
            loss = -model.forward(features, labels, mask)
            # TODO: Backpropogate the loss through our model
            loss.backward()
            # TODO: Update our coefficients in the direction of the gradient.
            optimizer.step()
            # TODO: Store the losses for logging
            losses.append(loss.item())
        # TODO: Log the average Loss for the epoch
        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        # TODO: make dev predictions with the `predict()` function
        dev_f1 = f1_score(predict(model, dev_features), dev_labels, outside_tag_idx=labels2i['O'])
        # TODO: Compute F1 score on the dev set and log it.
        print(f"Dev F1 {dev_f1}")
        
    # Return the trained model
    return model

In [ ]:
# Build the model and featurized data
train_features = featurize(train_sents)
dev_features = featurize(dev_sents)

# Get the full inventory of possible features
all_features = build_features_set(train_features)
# Hash all features to a unique int.
features_dict = make_features_dict(all_features)
# Initialize the model.
model = NERTagger(len(features_dict), len(labels2i))

encoded_train_features = encode_features(train_features, features_dict)
encoded_dev_features = encode_features(dev_features, features_dict)
encoded_train_labels = encode_labels(train_tag_sents, labels2i)
encoded_dev_labels = encode_labels(dev_tag_sents, labels2i)

# TODO: Play with hyperparameters here.
num_epochs = 30
batch_size = 16
LR=0.05
optimizer = torch.optim.SGD(model.parameters(), LR)

model = training_loop(
    num_epochs,
    batch_size,
    encoded_train_features,
    encoded_train_labels,
    encoded_dev_features,
    encoded_dev_labels,
    optimizer,
    model,
    labels2i,
    features_dict[PAD_SYMBOL]
)